# Download Data from GCP to DBFZ

In [0]:
%ls /dbfs/FileStore/shared_uploads/data4

In [0]:
%cp -r /dbfs/FileStore/shared_uploads/data /dbfs/FileStore/shared_uploads/data4/

In [0]:
%cp -r /dbfs/FileStore/shared_uploads/data /dbfs/FileStore/shared_uploads/data5/

In [0]:
%cp -r /dbfs/FileStore/shared_uploads/data /dbfs/FileStore/shared_uploads/data6/

In [0]:
%cp -r /dbfs/FileStore/shared_uploads/data /dbfs/FileStore/shared_uploads/data7/

In [0]:
%cp -r /dbfs/FileStore/shared_uploads/data /dbfs/FileStore/shared_uploads/data8/

In [0]:
%cp -r /dbfs/FileStore/shared_uploads/data /dbfs/FileStore/shared_uploads/data9/

In [0]:
%cp -r /dbfs/FileStore/shared_uploads/data /dbfs/FileStore/shared_uploads/data10/

In [0]:
%pip install --upgrade google-cloud-storage
dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached google_cloud_storage-2.13.0-py2.py3-none-any.whl (121 kB)
  Using cached google_resumable_media-2.6.0-py2.py3-none-any.whl (80 kB)
  Using cached google_api_core-2.14.0-py3-none-any.whl (122 kB)
  Using cached google_crc32c-1.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (32 kB)
  Using cached google_auth-2.25.0-py2.py3-none-any.whl (184 kB)
  Using cached google_cloud_core-2.3.3-py2.py3-none-any.whl (29 kB)
  Using cached protobuf-4.25.1-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
  Using cached pyasn1_modules-0.3.0-py2.py3-none-any.whl (181 kB)
  Using cached rsa-4.9-py3-none-any.whl (34 kB)
  Using cached cachetools-5.3.2-py3-none-any.whl (9.3 kB)
  Using cached pyasn1-0.5.1-py2.py3-none-any.whl (84 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.4
    Not uninstalling protobuf at /databricks/python3/lib/python3.1

In [0]:
from pyspark.sql import SparkSession
from google.cloud import storage
import os
from time import time
import json
from pyspark.dbutils import DBUtils

from pathlib import Path

# Initialize Spark Session
spark = SparkSession.builder.appName("GCSFilesDownload").getOrCreate()
sc = spark.sparkContext

dbutils = DBUtils(spark)

# Google Cloud credentials and bucket details
json_key_file_path = "../../dbfs/FileStore/shared_uploads/bubbly_sandbox_406100_b00687afd8a0.json"
gcs_bucket_name = "data516project"
gcs_blob_name = "data516project/Military Aircraft Detection/data/dataset"  # File path in GCS

# Authenticate and initialize GCS client
storage_client = storage.Client.from_service_account_json(json_key_file_path)
bucket = storage_client.get_bucket(gcs_bucket_name)

# List blobs in the GCS bucket
blobs = list(bucket.list_blobs(prefix='Military Aircraft Detection/data/dataset'))
filtered_blobs = [blob.name for blob in blobs if not (blob.name.endswith("/") or blob.name.endswith(".DS_Store") or blob.name.endswith(".csv"))]

In [0]:
# Define function to download a file
def download_blob(blob_name):
    storage_client = storage.Client.from_service_account_info(
        {
            "type": "service_account",
            "project_id": "bubbly-sandbox-406100",
            "private_key_id": "b00687afd8a0557d2daa2281fdf8a9ca4667b171",
            "private_key": "-----END PRIVATE KEY-----\n",
            "client_email": "452869396641-compute@developer.gserviceaccount.com",
            "client_id": "111941241038080001862",
            "auth_uri": "https://accounts.google.com/o/oauth2/auth",
            "token_uri": "https://oauth2.googleapis.com/token",
            "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
            "client_x509_cert_url": "https://www.googleapis.com/robot/v1/metadata/x509/452869396641-compute%40developer.gserviceaccount.com",
            "universe_domain": "googleapis.com"
        })
    bucket = storage_client.get_bucket(gcs_bucket_name)
    blob = bucket.blob(blob_name)
    local_path = '/dbfs/FileStore/shared_uploads/data_parallel/' + blob_name.split('/')[-1]
    blob.download_to_filename(local_path)
    return local_path

# Parallelize the download process
time_start = time()
rdd = sc.parallelize(filtered_blobs)
rdd.map(download_blob).collect()

# Log the time taken
print(f'Download Process took {time() - time_start}s for approximately {len(filtered_blobs)} files')

# Stop the Spark session
spark.stop()